In [1]:
# Setting up Imports 

import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Setting up Model Thresholds

model_memory_threshold = 60 # in KB

In [3]:
# Setting up Model Directories

models_dir = '../models/session1'
models = os.listdir(models_dir)
models

['2.keras', '0.keras', '1.keras']

In [24]:
# Get data

def get_data(input_shape):
    sample_nums = 1
    dims = [sample_nums]
    dims = dims + list(input_shape[:-1])
    data = tf.random.normal(shape = dims)
    data = (np.expand_dims(data, axis=-1))
    return data

In [25]:
# Model querying

for i in models:
    model = tf.keras.models.load_model(os.path.join(models_dir, i))
    input_dims = (model.layers[0].input_shape[0])[1:]
    data = get_data(input_dims)
    total = 0
    for i in range(1000):
        data = get_data(input_dims)
        tf.config.experimental.reset_memory_stats('GPU:0')
        model.predict(data, verbose=None)
        results = tf.config.experimental.get_memory_info('GPU:0')
        results = (results["peak"] - results["current"])/1000
        total = results + total
    results = total / 1000
    print(f"Memory used:",results,"KB")

Memory used: 157524.04070400292 KB
Memory used: 135841.83347199878 KB
Memory used: 135858.62681600108 KB


In [19]:
tf.config.experimental.reset_memory_stats('GPU:0')
tf.config.experimental.get_memory_info('GPU:0')

{'current': 249048320, 'peak': 249048320}

In [28]:
from tensorflow import lite
model = tf.keras.models.load_model(os.path.join(models_dir, models[0]))
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

    # Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpipz5nuxm/assets


INFO:tensorflow:Assets written to: /tmp/tmpipz5nuxm/assets


In [29]:
model = tf.keras.models.load_model("model.tflite")
data = get_data(input_dims)
total = 0
for i in range(1000):
    data = get_data(input_dims)
    tf.config.experimental.reset_memory_stats('GPU:0')
    model.predict(data, verbose=None)
    results = tf.config.experimental.get_memory_info('GPU:0')
    results = (results["peak"] - results["current"])/1000
    total = results + total
results = total / 1000
print(f"Memory used:",results,"KB")

OSError: Unable to open file (file signature not found)

In [30]:
class TensorflowLiteClassificationModel:
    def __init__(self, model_path, labels, image_size=224):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self._input_details = self.interpreter.get_input_details()
        self._output_details = self.interpreter.get_output_details()
        self.labels = labels
        self.image_size=image_size

    def run_from_filepath(self, image_path):
        input_data_type = self._input_details[0]["dtype"]
        image = np.array(Image.open(image_path).resize((self.image_size, self.image_size)), dtype=input_data_type)
        if input_data_type == np.float32:
            image = image / 255.

        if image.shape == (1, 224, 224):
            image = np.stack(image*3, axis=0)

        return self.run(image)

    def run(self, image):
        """
        args:
          image: a (1, image_size, image_size, 3) np.array

        Returns list of [Label, Probability], of type List<str, float>
        """

        self.interpreter.set_tensor(self._input_details[0]["index"], image)
        self.interpreter.invoke()
        tflite_interpreter_output = self.interpreter.get_tensor(self._output_details[0]["index"])
        probabilities = np.array(tflite_interpreter_output[0])

        # create list of ["label", probability], ordered descending probability
        label_to_probabilities = []
        for i, probability in enumerate(probabilities):
            label_to_probabilities.append([self.labels[i], float(probability)])
        return sorted(label_to_probabilities, key=lambda element: element[1])